In [25]:
import sqlite3
import pandas as pd

try:
    # Подключение к базе данных
    conn = sqlite3.connect('UEFA_ChampionsLeague_db_project_final_v.db')
    cursor = conn.cursor()

    # Включить проверку внешних ключей
    cursor.execute('PRAGMA foreign_keys = ON')
# Создание таблиц с необходимыми ограничениями

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Sponsors(
            Sponsor_ID TEXT PRIMARY KEY,
            Sponsor_Name TEXT NOT NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Coaches(
            Coach_ID TEXT PRIMARY KEY,
            Coach_Full_Name TEXT NOT NULL,
            DOB TEXT CHECK (DOB < DATE('now')),
            Team_Name TEXT,
            FOREIGN KEY (Team_Name) REFERENCES Teams(Team_Name) ON DELETE SET NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Referees(
            Referee_ID TEXT PRIMARY KEY,
            Referee_Name TEXT NOT NULL,
            Nationality TEXT NOT NULL,
            Experience_Years INTEGER CHECK (Experience_Years > 0),
            Matches_numbers INTEGER CHECK (Matches_numbers >= 0),
            Yellow_Cards INTEGER CHECK (Yellow_Cards >= 0),
            Red_Cards INTEGER CHECK (Red_Cards >= 0)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Tournaments(
            Tournament_ID TEXT PRIMARY KEY,
            Tournament_Name TEXT NOT NULL,
            Year INTEGER CHECK (Year <= CAST(strftime('%Y', 'now') AS INTEGER)),
            Winner TEXT,
            Location TEXT,
            Number_of_Teams INTEGER CHECK (Number_of_Teams > 0),
            Sponsor_ID TEXT,
            FOREIGN KEY (Sponsor_ID) REFERENCES Sponsors(Sponsor_ID) ON DELETE SET NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Teams(
            Team_ID TEXT PRIMARY KEY,
            Team_Name TEXT UNIQUE NOT NULL,
            Coach_ID TEXT,
            Sponsor_ID TEXT,
            Stadium_ID TEXT,
            Foundation_year INTEGER CHECK (Foundation_year <= CAST(strftime('%Y', 'now') AS INTEGER)),
            FOREIGN KEY (Coach_ID) REFERENCES Coaches(Coach_ID) ON DELETE SET NULL,
            FOREIGN KEY (Sponsor_ID) REFERENCES Sponsors(Sponsor_ID) ON DELETE SET NULL,
            FOREIGN KEY (Stadium_ID) REFERENCES Stadiums(Stadium_ID) ON DELETE SET NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Stadiums(
            Stadium_ID TEXT PRIMARY KEY,
            Stadium_Name TEXT NOT NULL,
            Location TEXT,
            City TEXT,
            Country TEXT NOT NULL,
            Capacity INTEGER CHECK (Capacity > 0),
            Year_Built INTEGER CHECK (Year_Built <= CAST(strftime('%Y', 'now') AS INTEGER)),
            Team_ID TEXT,
            FOREIGN KEY (Team_ID) REFERENCES Teams(Team_ID) ON DELETE SET NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Players(
            Player_ID TEXT PRIMARY KEY,
            Player_Name TEXT NOT NULL,
            Nationality TEXT NOT NULL,
            DOB TEXT CHECK (DOB < DATE('now')),
            Team_ID TEXT,
            Jersey_Number INTEGER CHECK (Jersey_Number >= 0),
            Game_Position TEXT,
            Height REAL CHECK (Height > 0),
            Weight REAL CHECK (Weight > 0),
            Foot TEXT CHECK (Foot IN ('L', 'R')),
            Salary REAL CHECK (Salary >= 0),
            Goals INTEGER CHECK (Goals >= 0),
            Assists INTEGER CHECK (Assists >= 0),
            Transfer_Year INTEGER CHECK (Transfer_Year <= CAST(strftime('%Y', 'now') AS INTEGER)),
            FOREIGN KEY (Team_ID) REFERENCES Teams(Team_ID) ON DELETE SET NULL,
            UNIQUE (Team_ID, Jersey_Number)
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Matches(
            Match_ID TEXT PRIMARY KEY,
            Penalty_Shoot_Out INTEGER CHECK (Penalty_Shoot_Out IN (0, 1)),
            Tournament_ID TEXT,
            Referee_ID TEXT,
            FOREIGN KEY (Tournament_ID) REFERENCES Tournaments(Tournament_ID) ON DELETE SET NULL,
            FOREIGN KEY (Referee_ID) REFERENCES Referees(Referee_ID) ON DELETE SET NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS Goals(
            Goal_ID TEXT PRIMARY KEY,
            Goal_Desc TEXT,
            Scored_ID TEXT,
            Match_Time INTEGER CHECK (Match_Time > 0),
            Assist_ID TEXT,
            Match_ID TEXT,
            FOREIGN KEY (Match_ID) REFERENCES Matches(Match_ID) ON DELETE SET NULL,
            FOREIGN KEY (Scored_ID) REFERENCES Players(Player_ID) ON DELETE SET NULL,
            FOREIGN KEY (Assist_ID) REFERENCES Players(Player_ID) ON DELETE SET NULL
        )
    ''')


    # Загрузка данных из Excel
    df_dict = pd.read_excel('UEFA Champions League 2016-2022 Data_27-12_final (1) (3).xlsx', sheet_name=None)

    # Define the order of processing to respect foreign key dependencies
    processing_order = [
        'Sponsors',
        'Coaches',
        'Referees',
        'Tournaments',
        'Teams',
        'Stadiums',
        'Players',
        'Matches',
        'Goals'
    ]

    for sheet_name in processing_order:
        if sheet_name not in df_dict:
            print(f"Лист {sheet_name} отсутствует в Excel-файле.")
            continue

        df = df_dict[sheet_name]
        print(f"Обрабатываем лист: {sheet_name}")

        # Замена NaN на None
        df = df.where(pd.notnull(df), None)

        # Удаление лишних пробелов
        # Replace applymap with apply and column-wise strip
        df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

        # Преобразование дат в правильный формат
        if 'DOB' in df.columns:
            df['DOB'] = pd.to_datetime(df['DOB'], errors='coerce').dt.strftime('%Y-%m-%d')

        try:
            if sheet_name == 'Sponsors':
                for _, row in df.iterrows():
                    cursor.execute('''
                        INSERT OR IGNORE INTO Sponsors (
                            Sponsor_ID, Sponsor_Name, Industry, Budget
                        ) VALUES (?, ?, ?, ?)
                    ''', (
                        row.get('Sponsor_ID'),
                        row.get('Sponsor_Name'),
                        row.get('Industry'),
                        row.get('Budget')
                    ))

            elif sheet_name == 'Coaches':
                for _, row in df.iterrows():
                    cursor.execute('''
                        INSERT OR IGNORE INTO Coaches (
                            Coach_ID, Coach_Full_Name, First_Name, Last_Name, Nationality, DOB, Team_Name
                        ) VALUES (?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        row.get('Coach_ID'),
                        row.get('Coach_Full_Name'),
                        row.get('First_Name'),
                        row.get('Last_Name'),
                        row.get('Nationality'),
                        row.get('DOB'),
                        row.get('Team_Name')
                    ))

            elif sheet_name == 'Referees':
                for _, row in df.iterrows():
                    cursor.execute('''
                        INSERT OR IGNORE INTO Referees (
                            Referee_ID, Referee_Name, Nationality, Experience_Years, Matches_numbers,
                            Yellow_Cards, Red_Cards
                        ) VALUES (?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        row.get('Referee_ID'),
                        row.get('Referee_Name'),
                        row.get('Nationality'),
                        row.get('Experience_Years'),
                        row.get('Matches_numbers'),
                        row.get('Yellow_Cards'),
                        row.get('Red_Cards')
                    ))

            elif sheet_name == 'Tournaments':
                for _, row in df.iterrows():
                    cursor.execute('''
                        INSERT OR IGNORE INTO Tournaments (
                            Tournament_ID, Tournament_Name, Year, Winner, Location, Number_of_Teams, Sponsor_ID
                        ) VALUES (?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        row.get('Tournament_ID'),
                        row.get('Tournament_Name'),
                        row.get('Year'),
                        row.get('Winner'),
                        row.get('Location'),
                        row.get('Number_of_Teams'),
                        row.get('Sponsor_ID')
                    ))

            elif sheet_name == 'Teams':
                # Ensure that Coaches and Sponsors are already inserted
                valid_coaches = set()
                if 'Coaches' in df_dict:
                    valid_coaches = set(df_dict['Coaches']['Coach_ID'].dropna().unique())

                valid_sponsors = set()
                if 'Sponsors' in df_dict:
                    valid_sponsors = set(df_dict['Sponsors']['Sponsor_ID'].dropna().unique())

                for _, row in df.iterrows():
                    coach_id = row.get('Coach_ID')
                    sponsor_id = row.get('Sponsor_ID')
                    # Validate foreign keys
                    if coach_id not in valid_coaches:
                        coach_id = None
                    if sponsor_id not in valid_sponsors:
                        sponsor_id = None

                    cursor.execute('''
                        INSERT OR IGNORE INTO Teams (
                            Team_ID, Team_Name, Main_Owner, Budget, Trophy_numbers,
                            Coach_ID, Sponsor_ID, Stadium_ID, Foundation_Year
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        row.get('Team_ID'),
                        row.get('Team_Name'),
                        row.get('Main_Owner'),
                        row.get('Budget'),
                        row.get('Trophy_numbers'),
                        coach_id,
                        sponsor_id,
                        row.get('Stadium_ID'),  # This will be NULL initially
                        row.get('Foundation_Year')
                    ))

            elif sheet_name == 'Stadiums':
                for _, row in df.iterrows():
                    cursor.execute('''
                        INSERT OR IGNORE INTO Stadiums (
                            Stadium_ID, Stadium_Name, Location, City, Country, Capacity, Year_Built
                        ) VALUES (?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        row.get('Stadium_ID'),
                        row.get('Stadium_Name'),
                        row.get('Location'),
                        row.get('City'),
                        row.get('Country'),
                        row.get('Capacity'),
                        row.get('Year_Built')
                    ))

                # Optionally, update Teams with Stadium_ID after Stadiums are inserted
                for _, row in df.iterrows():
                    stadium_id = row.get('Stadium_ID')
                    team_id = row.get('Team_ID')  # Assuming Stadiums have Team_ID in data
                    if stadium_id and team_id:
                        cursor.execute('''
                            UPDATE Teams
                            SET Stadium_ID = ?
                            WHERE Team_ID = ?
                        ''', (stadium_id, team_id))

            elif sheet_name == 'Players':
                # Ensure Teams are inserted
                valid_teams = set(df_dict['Teams']['Team_ID'].dropna().unique())

                for _, row in df.iterrows():
                    team_id = row.get('Team_ID')
                    if team_id not in valid_teams:
                        team_id = None

                    cursor.execute('''
                        INSERT OR IGNORE INTO Players (
                            Player_ID, Player_Name, Nationality, DOB, Team_Name,
                            Team_ID, Jersey_Number, Game_Position, Height, Weight,
                            Foot, Salary, Goals, Assists, Transfer_Year
                        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
                    ''', (
                        row.get('Player_ID'),
                        row.get('Player_Name'),
                        row.get('Nationality'),
                        row.get('DOB'),
                        row.get('Team_Name'),
                        team_id,
                        row.get('Jersey_Number'),
                        row.get('Game_Position'),
                        row.get('Height'),
                        row.get('Weight'),
                        row.get('Foot'),
                        row.get('Salary'),
                        row.get('Goals'),
                        row.get('Assists'),
                        row.get('Transfer_Year')
                    ))

            elif sheet_name == 'Matches':
                # Ensure Tournaments and Referees are inserted
                valid_tournaments = set(df_dict['Tournaments']['Tournament_ID'].dropna().unique())
                valid_referees = set(df_dict['Referees']['Referee_ID'].dropna().unique())

                for _, row in df.iterrows():
                    tournament_id = row.get('Tournament_ID')
                    referee_id = row.get('Referee_ID')

                    if tournament_id not in valid_tournaments:
                        tournament_id = None
                    if referee_id not in valid_referees:
                        referee_id = None

                    cursor.execute('''
                        INSERT OR IGNORE INTO Matches (
                            Match_ID, Penalty_Shoot_Out, Tournament_ID, Referee_ID
                        ) VALUES (?, ?, ?, ?)
                    ''', (
                        row.get('Match_ID'),
                        row.get('Penalty_Shoot_Out'),
                        tournament_id,
                        referee_id
                    ))

            elif sheet_name == 'Goals':
                # Ensure Matches are inserted
                valid_matches = set(df_dict['Matches']['Match_ID'].dropna().unique())

                for _, row in df.iterrows():
                    match_id = row.get('Match_ID')
                    if match_id not in valid_matches:
                        match_id = None

                    cursor.execute('''
                        INSERT OR IGNORE INTO Goals (
                            Goal_ID, Goal_Desc, Scored_ID, Match_Time, Assist_ID, Match_ID
                        ) VALUES (?, ?, ?, ?, ?, ?)
                    ''', (
                        row.get('Goal_ID'),
                        row.get('Goal_Desc'),
                        row.get('Scored_ID'),
                        row.get('Match_Time'),
                        row.get('Assist_ID'),
                        match_id
                    ))

        except Exception as e:
            print(f"Ошибка при обработке листа {sheet_name}: {e}")

    conn.commit()

except sqlite3.Error as e:
    print(f"Произошла ошибка SQLite: {e}")

finally:
    if conn:
        conn.close()


Обрабатываем лист: Sponsors
Обрабатываем лист: Coaches
Обрабатываем лист: Referees
Обрабатываем лист: Tournaments
Обрабатываем лист: Teams
Обрабатываем лист: Stadiums
Обрабатываем лист: Players
Обрабатываем лист: Matches
Обрабатываем лист: Goals
